<a href="https://colab.research.google.com/github/ahathaway821/COVID-19-Fact-Checking/blob/master/src/data-scraping/princeton_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load csv
import pandas as pd
import os
import re
from wordcloud import WordCloud
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import glob
import json

In [2]:
!git clone https://github.com/ahathaway821/COVID-19-Fact-Checking.git

Cloning into 'COVID-19-Fact-Checking'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 142 (delta 45), reused 73 (delta 17), pack-reused 0
Receiving objects: 100% (142/142), 4.99 MiB | 15.53 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [39]:
df = pd.read_csv('/content/COVID-19-Fact-Checking/data/esoc_4_June_2020.csv', encoding = "ISO-8859-1")
df.head()

,ID,Reported_On,Additional_Reporting,Retrieve_from_1,Retrieve_from_2,Retrieve_from_3,Twitter_Reference,Direct_Post_1,Direct_Post_2,Direct_Post_3,Direct_Post_4,Title,Publication_Date,Entry_Date,Country,Primary_Language,Secondary_Language,Main_Narrative,Narrative_Description,ChainMsg_or_Email,Motive,Motive_Description,Source,Source_Description,Distrib_Channel,Misinfo_Type,Key_Words,Summary,Coder,Unnamed: 29
0,1.0,https://www.buzzfeednews.com/article/ryanhates...,NaN,https://www.buzzfeednews.com/article/ryanhates...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,India Is In The Middle Of A Coronavirus YouTub...,19-Feb-20,16-Mar-20,India,English,NaN,Origin of the virus,Hindi language YouTube account saying COVID-19...,0.0,Politics,Efforts to spread false claims on the origins ...,Individual actor,General public,Youtube,Conspiracy,"Coronavirus, India, bat soup",Hindi language Youtube account suggesting COVI...,Jan,NaN
1,2.0,https://twitter.com/Rangoli_A/status/122779241...,NaN,NaN,NaN,NaN,1.0,https://twitter.com/Rangoli_A/status/122779241...,NaN,NaN,NaN,"Tweet: False reporting on ""shooting down"" peop...",12-Feb-20,16-Mar-20,Ambiguous,English,NaN,Emergency responses,Twitter user posting a compilation video of pe...,0.0,Fear,"Twitter user stoking fear among other users, s...",Individual actor,General public,Twitter,False reporting,"Coronavirus, China, shooting","Tweet with video showing ""people getting shot ...",Jan,NaN
2,3.0,https://twitter.com/Woppa1Woppa/status/1220068...,NaN,NaN,NaN,NaN,1.0,https://twitter.com/Woppa1Woppa/status/1220068...,NaN,NaN,NaN,"Tweet: A woman eating a delicacy, attributing ...",3-Feb-20,16-Mar-20,Ambiguous,English,NaN,Origin of the virus,Twitter user posting a video of a celebrity ea...,0.0,Politics,Twitter user discrediting Chinese-American pop...,Individual actor,General public,Twitter,False reporting,"Coronavirus, Chinese food, bat soup","Video of an individual eating a delicacy, and ...",Jan,NaN
3,4.0,https://twitter.com/FreddiGoldstein/status/123...,NaN,NaN,NaN,NaN,1.0,https://twitter.com/FreddiGoldstein/status/123...,NaN,NaN,NaN,Tweet: False chain message about NYPD containm...,12-Mar-20,16-Mar-20,US,English,NaN,Emergency responses,Chain message of NYPD containment responses.,1.0,Fear,Chain message spread to stoke fear among Ameri...,Individual actor,General public,"Media, SMS",False reporting,"Coronavirus, NYPD, containment zone",Tweet with a screenshot of chain message sugge...,Jan,NaN
4,5.0,https://www.boomlive.in/health/hoax-alert-vira...,NaN,https://www.boomlive.in/health/hoax-alert-vira...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Hoax Alert: Viral 'Emergency Notification' On ...,28-Jan-20,16-Mar-20,India,English,NaN,Emergency responses,Chain message of Indian Health Ministry respon...,1.0,Fear,Chain message spread to stoke fear among India...,Individual actor,General public,"Facebook, WhatsApp",False reporting,"Coronavirus, India, travel advisory",WhatsApp chain message circulating among India...,Jan,NaN


In [0]:
df['source_label'] = df['Misinfo_Type']

In [0]:
df.rename(columns={'Narrative_Description': 'claim', 
                   'Misinfo_Type': 'label',
                   'Entry_Date': 'date',
                   'Distrib_Channel': 'claim_source',
                   'Summary': 'explanation'}, inplace=True)

In [42]:
df.head()

,ID,Reported_On,Additional_Reporting,Retrieve_from_1,Retrieve_from_2,Retrieve_from_3,Twitter_Reference,Direct_Post_1,Direct_Post_2,Direct_Post_3,Direct_Post_4,Title,Publication_Date,date,Country,Primary_Language,Secondary_Language,Main_Narrative,claim,ChainMsg_or_Email,Motive,Motive_Description,Source,Source_Description,claim_source,label,Key_Words,explanation,Coder,Unnamed: 29,source_label
0,1.0,https://www.buzzfeednews.com/article/ryanhates...,NaN,https://www.buzzfeednews.com/article/ryanhates...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,India Is In The Middle Of A Coronavirus YouTub...,19-Feb-20,16-Mar-20,India,English,NaN,Origin of the virus,Hindi language YouTube account saying COVID-19...,0.0,Politics,Efforts to spread false claims on the origins ...,Individual actor,General public,Youtube,Conspiracy,"Coronavirus, India, bat soup",Hindi language Youtube account suggesting COVI...,Jan,NaN,Conspiracy
1,2.0,https://twitter.com/Rangoli_A/status/122779241...,NaN,NaN,NaN,NaN,1.0,https://twitter.com/Rangoli_A/status/122779241...,NaN,NaN,NaN,"Tweet: False reporting on ""shooting down"" peop...",12-Feb-20,16-Mar-20,Ambiguous,English,NaN,Emergency responses,Twitter user posting a compilation video of pe...,0.0,Fear,"Twitter user stoking fear among other users, s...",Individual actor,General public,Twitter,False reporting,"Coronavirus, China, shooting","Tweet with video showing ""people getting shot ...",Jan,NaN,False reporting
2,3.0,https://twitter.com/Woppa1Woppa/status/1220068...,NaN,NaN,NaN,NaN,1.0,https://twitter.com/Woppa1Woppa/status/1220068...,NaN,NaN,NaN,"Tweet: A woman eating a delicacy, attributing ...",3-Feb-20,16-Mar-20,Ambiguous,English,NaN,Origin of the virus,Twitter user posting a video of a celebrity ea...,0.0,Politics,Twitter user discrediting Chinese-American pop...,Individual actor,General public,Twitter,False reporting,"Coronavirus, Chinese food, bat soup","Video of an individual eating a delicacy, and ...",Jan,NaN,False reporting
3,4.0,https://twitter.com/FreddiGoldstein/status/123...,NaN,NaN,NaN,NaN,1.0,https://twitter.com/FreddiGoldstein/status/123...,NaN,NaN,NaN,Tweet: False chain message about NYPD containm...,12-Mar-20,16-Mar-20,US,English,NaN,Emergency responses,Chain message of NYPD containment responses.,1.0,Fear,Chain message spread to stoke fear among Ameri...,Individual actor,General public,"Media, SMS",False reporting,"Coronavirus, NYPD, containment zone",Tweet with a screenshot of chain message sugge...,Jan,NaN,False reporting
4,5.0,https://www.boomlive.in/health/hoax-alert-vira...,NaN,https://www.boomlive.in/health/hoax-alert-vira...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Hoax Alert: Viral 'Emergency Notification' On ...,28-Jan-20,16-Mar-20,India,English,NaN,Emergency responses,Chain message of Indian Health Ministry respon...,1.0,Fear,Chain message spread to stoke fear among India...,Individual actor,General public,"Facebook, WhatsApp",False reporting,"Coronavirus, India, travel advisory",WhatsApp chain message circulating among India...,Jan,NaN,False reporting


In [0]:
df['source'] = 'Princeton'

In [0]:
df['fact_check_url'] = ''

In [0]:
cols = ['claim', 'label', 'source_label', 'source', 'date', 'claim_source', 'explanation', 'fact_check_url']
df_new = df[['claim', 'label', 'source_label', 'source', 'date', 'claim_source', 'explanation', 'fact_check_url']]

In [46]:
df_new.head()

,claim,label,source_label,source,date,claim_source,explanation,fact_check_url
0,Hindi language YouTube account saying COVID-19...,Conspiracy,Conspiracy,Princeton,16-Mar-20,Youtube,Hindi language Youtube account suggesting COVI...,
1,Twitter user posting a compilation video of pe...,False reporting,False reporting,Princeton,16-Mar-20,Twitter,"Tweet with video showing ""people getting shot ...",
2,Twitter user posting a video of a celebrity ea...,False reporting,False reporting,Princeton,16-Mar-20,Twitter,"Video of an individual eating a delicacy, and ...",
3,Chain message of NYPD containment responses.,False reporting,False reporting,Princeton,16-Mar-20,"Media, SMS",Tweet with a screenshot of chain message sugge...,
4,Chain message of Indian Health Ministry respon...,False reporting,False reporting,Princeton,16-Mar-20,"Facebook, WhatsApp",WhatsApp chain message circulating among India...,


In [0]:
df_new = df_new[df_new['label'].notna()]

In [48]:
df_new.label.unique()

array(['Conspiracy', 'False reporting', 'Fake remedy', 'False remedy',
       'False reporting, Fake remedy', 'Conspiracy, False reporting',
       'False reporting, Conspiracy', 'Fake remedy, False reporting',
       'Conspiracy, Fake remedy', 'Fake remedy, conspiracy'], dtype=object)

In [0]:
df_new.loc[df_new.label == "Conspiracy", "label"] = "false"
df_new.loc[df_new.label == "False reporting", "label"] = "false"
df_new.loc[df_new.label == "Fake remedy", "label"] = "false"
df_new.loc[df_new.label == "False remedy", "label"] = "false"
df_new.loc[df_new.label == "False reporting, Fake remedy", "label"] = "false"
df_new.loc[df_new.label == "Conspiracy, False reporting", "label"] = "false"
df_new.loc[df_new.label == "False reporting, Conspiracy", "label"] = "false"
df_new.loc[df_new.label == "Fake remedy, False reporting", "label"] = "false"
df_new.loc[df_new.label == "Conspiracy, Fake remedy", "label"] = "false"
df_new.loc[df_new.label == "Fake remedy, conspiracy", "label"] = "false"

In [0]:
df_obj = df_new.select_dtypes(['object'])
df_new[df_obj.columns] = df_obj.apply(lambda x: x.str.strip('\r\n\t'))

In [0]:
df_obj = df_new.select_dtypes(['object'])
df_new[df_obj.columns] = df_obj.apply(lambda x: x.str.strip(''))

In [0]:
df_new['claim'] = df_new['claim'].str.strip('\r\n\t')
df_new['explanation'] = df_new['explanation'].str.strip('\r\n\t')

In [89]:
df_new.head()

,claim,label,source_label,source,date,claim_source,explanation,fact_check_url
0,Hindi language YouTube account saying COVID-19...,false,Conspiracy,Princeton,16-Mar-20,Youtube,Hindi language Youtube account suggesting COVI...,
1,Twitter user posting a compilation video of pe...,false,False reporting,Princeton,16-Mar-20,Twitter,"Tweet with video showing ""people getting shot ...",
2,Twitter user posting a video of a celebrity ea...,false,False reporting,Princeton,16-Mar-20,Twitter,"Video of an individual eating a delicacy, and ...",
3,Chain message of NYPD containment responses.,false,False reporting,Princeton,16-Mar-20,"Media, SMS",Tweet with a screenshot of chain message sugge...,
4,Chain message of Indian Health Ministry respon...,false,False reporting,Princeton,16-Mar-20,"Facebook, WhatsApp",WhatsApp chain message circulating among India...,


In [0]:
df_new.to_csv('cleaned_princeton_data.csv')